In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [13]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32
directory="Data_complete_task2"
categories=["Mask_Chin","Mask_Mouth_Chin","Mask_Nose_Mouth"]

In [14]:
print("[INFO] loading images...")

data = []
labels = []
for imagePath in categories:
    print('imagePath',imagePath)
    path=os.path.join(directory,imagePath)
    for img in os.listdir(path):
        img_path=os.path.join(path,img)
#         print('path',path,img_path)
        image = load_img(img_path, target_size=(48, 48))
        image = img_to_array(image)
        image = preprocess_input(image)
        data.append(image)
        labels.append(imagePath)

[INFO] loading images...
imagePath Mask_Chin
imagePath Mask_Mouth_Chin
imagePath Mask_Nose_Mouth


In [15]:
lb = LabelEncoder()
labels = lb.fit_transform(labels)
labels

array([0, 0, 0, ..., 2, 2, 2], dtype=int64)

In [16]:
labels = to_categorical(labels)
data = np.array(data, dtype="float32")
labels = np.array(labels)
labels

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [17]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)


aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest",
)
aug

In [18]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
                        input_tensor=Input(shape=(48, 48, 3)),
                       classes=3)
print(baseModel.output)
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(2, 2))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(48, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3, activation="softmax")(headModel)

KerasTensor(type_spec=TensorSpec(shape=(None, 2, 2, 1280), dtype=tf.float32, name=None), name='out_relu/Relu6:0', description="created by layer 'out_relu'")


In [19]:
model = Model(inputs=baseModel.input, outputs=headModel)
print(model)
for layer in baseModel.layers:
    layer.trainable = False

print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


C:\Users\Rajath.DESKTOP-ELKQ6GV\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [20]:
print("[INFO] training head...")
H = model.fit(aug.flow(trainX, trainY, batch_size=BS),steps_per_epoch=len(trainX) // BS, validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

[INFO] training head...
Epoch 1/20
626/626 [==============================] - 84s 120ms/step - loss: 1.0848 - accuracy: 0.5294 - val_loss: 0.8406 - val_accuracy: 0.6025
Epoch 2/20
626/626 [==============================] - 66s 106ms/step - loss: 0.8345 - accuracy: 0.6106
Epoch 3/20
626/626 [==============================] - 70s 112ms/step - loss: 0.7852 - accuracy: 0.6369
Epoch 4/20
626/626 [==============================] - 67s 107ms/step - loss: 0.7493 - accuracy: 0.6540
Epoch 5/20
626/626 [==============================] - 71s 112ms/step - loss: 0.7274 - accuracy: 0.6732
Epoch 6/20
626/626 [==============================] - 78s 123ms/step - loss: 0.7070 - accuracy: 0.6813
Epoch 7/20
626/626 [==============================] - 67s 107ms/step - loss: 0.6940 - accuracy: 0.6907
Epoch 8/20
626/626 [==============================] - 61s 97ms/step - loss: 0.6915 - accuracy: 0.6899
Epoch 9/20
626/626 [==============================] - 66s 105ms/step - loss: 0.6749 - accuracy: 0.6998
Epoch 10

In [21]:
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testY.argmax(axis=1), predIdxs,target_names=lb.classes_))

                 precision    recall  f1-score   support

      Mask_Chin       0.81      0.29      0.43       967
Mask_Mouth_Chin       0.69      0.89      0.78      2795
Mask_Nose_Mouth       0.73      0.60      0.66      1249

       accuracy                           0.70      5011
      macro avg       0.74      0.59      0.62      5011
   weighted avg       0.72      0.70      0.68      5011



In [10]:
print("[INFO] training head...")
H = model.fit(aug.flow(trainX, trainY, batch_size=BS),steps_per_epoch=len(trainX) // BS, validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

[INFO] training head...
Epoch 1/20
499/499 [==============================] - 79s 141ms/step - loss: 1.1628 - accuracy: 0.4501 - val_loss: 0.9279 - val_accuracy: 0.5483
Epoch 2/20
499/499 [==============================] - 56s 113ms/step - loss: 0.9029 - accuracy: 0.5572
Epoch 3/20
499/499 [==============================] - 61s 121ms/step - loss: 0.8398 - accuracy: 0.5980
Epoch 4/20
499/499 [==============================] - ETA: 0s - loss: 0.7972 - accuracy: 0.62 - 67s 135ms/step - loss: 0.7972 - accuracy: 0.6250
Epoch 5/20
499/499 [==============================] - 53s 107ms/step - loss: 0.7788 - accuracy: 0.6353
Epoch 6/20
499/499 [==============================] - 48s 96ms/step - loss: 0.7469 - accuracy: 0.6571
Epoch 7/20
499/499 [==============================] - 66s 132ms/step - loss: 0.7363 - accuracy: 0.6673
Epoch 8/20
499/499 [==============================] - 59s 117ms/step - loss: 0.7176 - accuracy: 0.6767
Epoch 9/20
499/499 [==============================] - 53s 106ms/step 

In [11]:
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testY.argmax(axis=1), predIdxs,target_names=lb.classes_))

                 precision    recall  f1-score   support

      Mask_Chin       0.80      0.43      0.56       967
Mask_Mouth_Chin       0.63      0.81      0.71      1778
Mask_Nose_Mouth       0.74      0.70      0.72      1249

       accuracy                           0.68      3994
      macro avg       0.72      0.65      0.66      3994
   weighted avg       0.71      0.68      0.68      3994



In [15]:
print("[INFO] training head...")
H = model.fit(aug.flow(trainX, trainY, batch_size=BS),steps_per_epoch=len(trainX) // BS, validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

[INFO] training head...
Epoch 1/20
1668/1668 [==============================] - 205s 120ms/step - loss: 0.6094 - accuracy: 0.8123 - val_loss: 0.4820 - val_accuracy: 0.8358
Epoch 2/20
1668/1668 [==============================] - 170s 102ms/step - loss: 0.4911 - accuracy: 0.8330 - loss: 0.4911 - accuracy: 0.
Epoch 3/20
1668/1668 [==============================] - 174s 104ms/step - loss: 0.4606 - accuracy: 0.8383
Epoch 4/20
1668/1668 [==============================] - 176s 105ms/step - loss: 0.4476 - accuracy: 0.8414
Epoch 5/20
1668/1668 [==============================] - 178s 107ms/step - loss: 0.4373 - accuracy: 0.8430
Epoch 6/20
1668/1668 [==============================] - 170s 102ms/step - loss: 0.4269 - accuracy: 0.8460
Epoch 7/20
1668/1668 [==============================] - 174s 104ms/step - loss: 0.4215 - accuracy: 0.8452
Epoch 8/20
1668/1668 [==============================] - 167s 100ms/step - loss: 0.4166 - accuracy: 0.8480
Epoch 9/20
1668/1668 [==============================] - 

In [16]:
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testY.argmax(axis=1), predIdxs,target_names=lb.classes_))

                 precision    recall  f1-score   support

      Mask_Chin       0.84      0.05      0.09       967
Mask_Mouth_Chin       0.85      0.99      0.92     11131
Mask_Nose_Mouth       0.77      0.19      0.31      1249

       accuracy                           0.85     13347
      macro avg       0.82      0.41      0.44     13347
   weighted avg       0.84      0.85      0.80     13347

